### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Parkinsons_Disease/GSE202665'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for the availability of gene expression data
is_gene_available = True

# Setting the rows for available data
trait_row = 0  # 'disease state: Parkinson's disease', 'disease state: Healthy Control'
age_row = 3    # 'age: 53', 'age: 57', 'age: 63', 'age: 75', 'age: 85', 'age: 76', 'age: 69', 'age: 66'
gender_row = 2  # 'gender: male'

# Define the data type conversion functions
def convert_trait(value):
    if "Parkinson's disease" in value:
        return 1
    elif 'Healthy Control' in value:
        return 0
    return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    if 'male' in value:
        return 1
    elif 'female' in value:
        return 0
    return None

# Save cohort information
save_cohort_info('GSE202665', './preprocessed/Parkinsons_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical features and save to CSV if clinical data is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Parkinsons_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Parkinsons_Disease/trait_data/GSE202665.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# Inspect the output of STEP3 to determine if identifiers are gene symbols
identifiers = [
    '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'
]

# Based on biomedical knowledge, gene symbols are usually alphanumeric symbols,
# not purely numeric. Hence, these likely require mapping.


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
if requires_gene_mapping:
    # 1. Determine the key storing identifiers and the key storing gene symbols
    identifier_key = 'NAME'
    gene_symbol_key = 'GENE_SYMBOL'
    
    # 2. Get the dataframe storing the mapping between probe IDs and genes
    mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    # Adjusting columns name 'ID' to effective identifier_key inferred
    mapping_df = mapping_df.rename(columns={identifier_key: 'ID'}) # Ensure to rename in the dataframe

    # 3. Apply the mapping to get the transformed gene_data
    gene_data = apply_gene_mapping(gene_data, mapping_df)
